<a href="https://colab.research.google.com/github/ummeamunira/NLP-LLM/blob/main/reranker/Reranker_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

# Load pretrained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Example data: list of query-document pairs
queries = ["oil leak detection"]
documents = ["Detected an oil leak in the main pipeline.", "Regular maintenance required for the pump.", "Environmental issue due to oil spill."]

# Tokenize and encode the query-document pairs
inputs = tokenizer(queries * len(documents), documents, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor([1, 0, 1])  # Example relevance labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
labels

tensor([1, 0, 1])

In [5]:


# Create DataLoader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2)

# Train the model (simplified)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
model.train()
for batch in dataloader:
    input_ids, attention_mask, labels = batch
    optimizer.zero_grad()
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Use the model to rerank documents
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    scores = outputs.logits

# Print reranked documents based on scores
reranked_indices = scores[:, 1].argsort(descending=True)
reranked_documents = [documents[i] for i in reranked_indices]
print(reranked_documents)


['Detected an oil leak in the main pipeline.', 'Regular maintenance required for the pump.', 'Environmental issue due to oil spill.']
